# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [61]:

import pandas as pd

#загрузка данных из файла
recipes = pd.read_csv('recipes_sample.csv')

#выбор 5 случайных рецептов
random_recipes = recipes.sample(n=5)

#форматирование строк для вывода таблицы
table_format = '|{:^10}|{:^10}|'
header = table_format.format('id', 'minutes')
separator = '-' * len(header)
rows = [table_format.format(row['id'], row['minutes']) for _, row in random_recipes.iterrows()]
table = '\n'.join([header, separator] + rows)

# Вывод таблицы на экран
print(table)

|    id    | minutes  |
-----------------------
|  171512  |    10    |
|  170753  |    50    |
|  78700   |    50    |
|  45169   |    40    |
|  490274  |    60    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [23]:
import xml.etree.ElementTree as ET

#чтение файла с рецептами
with open('recipes_sample.csv', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    recipes = list(reader)

#чтение файла с шагами приготовления
tree = ET.parse('steps_sample.xml')
root = tree.getroot()

def show_info(recipe_id):
    #поиск рецепта по id
    recipe = next((r for r in recipes if r['id'] == recipe_id), None)
    if recipe:
        #формирование заголовка
        title = recipe['name'].title()
        #формирование списка шагов
        steps = []
        for step in root.findall(f"./recipe[@id='{recipe_id}']/step"):
            steps.append(step.text.strip())
        #формирование строки с информацией о рецепте
        info = f"{title}\n\n"
        for i, step in enumerate(steps):
            info += f"{i+1}. {step}\n"
        info += "----------\n"
        info += f"Автор: {recipe['contributor_id']}\n"
        info += f"Среднее время приготовления: {recipe['minutes']} минут"
        return info
    else:
        return "Рецепт не найден"

#вывод информации о рецепте с id 170895
print(show_info('170895'))

Leeks And Parsnips  Sauteed Or Creamed

----------
Автор: 8377
Среднее время приготовления: 27 минут


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
import re

with open('data/steps_sample.xml', 'r') as f:
    data = f.read()

recipe_id = '25082'
pattern = r'\d+\s(hour|hours|minute|minutes)'

recipe_steps = re.findall(f'<recipe>\s*<id>{recipe_id}</id>\s*<steps>(.*?)</steps>', data, flags=re.DOTALL)
if recipe_steps:
    recipe_steps = recipe_steps[0]
    matches = re.findall(pattern, recipe_steps)
    for match in matches:
        print(match)

minute
minute
hour
minute
minute
minute


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [19]:
import pandas as pd

df = pd.read_xml('data/steps_sample.xml', xpath='/recipes/recipe')
descriptions = df['steps'].astype(str).str.join(' ').str.lower()

pattern = "^this\.{3},?\s*but"
mask = descriptions.str.contains(pattern)

num_matches = mask.sum()
print(f"Количество рецептов, содержащих шаблон: {num_matches}\n")

sample_matches = df[mask].head(3)
for index, row in sample_matches.iterrows():
    print(f"Рецепт {index}:")
    print(row['steps'], "\n")

Количество рецептов, содержащих шаблон: 0



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [24]:
import xml.etree.ElementTree as ET

tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

recipe_id = '72367'
steps = []

for recipe in root.findall('recipe'):
    if recipe.find('id').text == recipe_id:
        for step in recipe.find('steps').findall('step'):
            steps.append(step.text)

print(steps)

for i in range(len(steps)):
    steps[i] = re.sub(r'\s*(\d+\s*/\s*\d+)\s*', r'\1', steps[i])

print(steps)


['mix butter , flour , 1 / 3 c', 'sugar and 1-1 / 4 t', 'vanilla', 'press into greased 9" springform pan', 'mix cream cheese , 1 / 4 c', 'sugar , eggs and 1 / 2 t', 'vanilla beating until fluffy', 'pour over dough', 'combine apples , 1 / 3 c', 'sugar and cinnamon', 'arrange on top of cream cheese mixture and sprinkle with almonds', 'bake at 350 for 45-55 minutes , or until tester comes out clean']
['mix butter , flour ,1 / 3c', 'sugar and 1-1 / 4t', 'vanilla', 'press into greased 9" springform pan', 'mix cream cheese ,1 / 4c', 'sugar , eggs and1 / 2t', 'vanilla beating until fluffy', 'pour over dough', 'combine apples ,1 / 3c', 'sugar and cinnamon', 'arrange on top of cream cheese mixture and sprinkle with almonds', 'bake at 350 for 45-55 minutes , or until tester comes out clean']


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from xml.etree import ElementTree

tree = ElementTree.parse('data/steps_sample.xml')
root = tree.getroot()
steps = []
for recipe in root.findall('recipe'):
    for step in recipe.find('steps'):
        steps.append(step.text)

unique_words = set()
for step in steps:
    tokens = word_tokenize(step)
    for token in tokens:
        if token.isalpha():
            unique_words.add(token.lower())

print(len(unique_words))




14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [3]:
import xml.etree.ElementTree as ET
import nltk

tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

sentences = []
for recipe in root.findall('recipe'):
    for step in recipe.find('steps'):
        sentences.extend(nltk.sent_tokenize(step.text.strip()))


words = [nltk.word_tokenize(sent) for sent in sentences]

lemmatizer = nltk.WordNetLemmatizer()
lemmatized_words = [[lemmatizer.lemmatize(word) for word in sent] for sent in words]

recipes = []
for recipe in root.findall('recipe'):
    steps = recipe.find('steps')
    num_sentences = sum([len(nltk.sent_tokenize(step.text.strip())) for step in steps])
    recipes.append((recipe.find('id').text, num_sentences))

recipes.sort(key=lambda x: x[1], reverse=True)

for recipe in recipes[:5]:
    for r in root.findall('recipe'):
        if r.find('id').text == recipe[0]:
            print(f"Id рецепта: {recipe[0]}, Количество предложений: {recipe[1]}")
            for step in r.find('steps'):
                print(step.text.strip())
            print('\n')


Id рецепта: 337926, Количество предложений: 95
pie pumpkins "are smaller , sweeter , less grainy textured pumpkins than the usual jack-o-lantern types
grocery stores usually carry them in late september through december in the u
s
they're only about 6 to 8 inches in diameter
if you're in a pinch and can't find a pie pumpkin , here's a
just like selecting any squash , look for one that is firm , no bruises or soft spots , and a good orange color
one 6" pie pumpkin usually makes one 10 inch deep dish pie and a bit extra
or 2 9 inch shallow pies !
wash the exterior of the pumpkin in cool or warm water , no soap
cut the pumpkin in half
a serrated knife and a sawing motion works best - a smooth knife is more likely to slip and hurt you ! a visitor suggests using a hand saw
and scrape the insides
you want to get out that stringy , dangly stuff that coats the inside surface
i find a heavy ice cream scoop works great for this
the seeds can be used either to plant pumpkins next year , or roaste

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [57]:
# функция для вывода информации о частях речи слов в предложении
def pos_tag_sentence(sentence):
    # разбиение предложения на слова
    words = nltk.word_tokenize(sentence)
    # определение частей речи для каждого слова
    pos_tags = nltk.pos_tag(words)
    # форматирование строки с информацией о частях речи слов
    formatted_tags = ' '.join(['{:<10}'.format(tag[1]) for tag in pos_tags])
    formatted_words = ' '.join(['{:<10}'.format(word) for word in words])
    # вывод информации о частях речи слов
    print(formatted_tags)
    print(formatted_words)

# пример использования функции на названии рецепта с id 241106
title = recipes.loc[recipes['id'] == 241106, 'description'].iloc[0]
pos_tag_sentence(title)

DT         VBP        DT         RB         JJ         NN         NN         TO         VB         IN         DT         NN         WRB        JJ         NNS        VBP        JJ         NN         CC         NNS        VBG        IN         .          DT         VBG        VBN        VBZ        DT         NN         ,          CC         EX         VBP        RB         JJ         JJR        NNS        TO         VB         DT         NN         .          VB         IN         IN         DT         JJ         NNS        CC         RB         VB         IN         WP         VBZ        JJ         TO         PRP        .         
these      are        a          really     good       quick      meal       to         make       in         the        summertime when       local      farms      have       fresh      eggplant   and        tomatoes   coming     in         .          the        topping    given      is         a          suggestion ,          but        there      are       